Julia Kang - jkang613@gmail.com - AMLI 2019 - Mills College

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
# from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from math import sqrt #for RMSE
  
import tensorflow as tf 
from tensorflow.feature_column import numeric_column,categorical_column_with_hash_bucket,categorical_column_with_vocabulary_list
from tensorflow.estimator import LinearClassifier
from tensorflow import keras
from tensorflow.data import Dataset

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Model: Multilayer Perception Neural Network

In [4]:
# raw_df = pd.read_csv('numeric_rand_sample.csv')
raw_df = pd.read_csv('choice_features.csv')
raw_df = raw_df.drop('Unnamed: 0', axis = 1) 

raw_df.head() #ad id is unique to each row
len(raw_df)

4184

In [5]:
FEATURES = []

for col in raw_df.columns:
    if np.issubdtype(raw_df[col].dtypes , np.number):
        FEATURES.append(col)
FEATURES.remove('click')
TARGET = 'click'

In [7]:
#FEATURES

In [8]:
# split the data into test and train
train, test  = train_test_split(
  raw_df,                        # split our digits dataframe
  stratify=raw_df[TARGET],       # stratify by the 'click' column   
  test_size=0.2,                # 20% of the data should be held for testing
  random_state=42               # hard-coded random state for repeatability in the example
)

train.head()

,click,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_id,device_ip,device_model,...,C15,C16,date,time,user_id,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
4114,0,1005,0,225,4,3,12,438,931,406,...,320,50,9,18,963,1499,1499,1499,170,148
1250,0,1002,0,274,4,12,0,266,2147,564,...,320,50,2,14,1724,1,1,1,211,232
3862,0,1005,1,83,12,12,0,438,1613,123,...,320,50,9,7,1694,24,24,24,174,232
2094,1,1005,0,275,1,12,0,438,451,864,...,320,50,5,2,479,720,720,720,136,232
670,1,1005,0,225,4,7,2,543,922,67,...,320,50,1,10,3449,1499,1499,1499,232,248


In [9]:
feature_columns = []

for col in FEATURES:
    feature_columns.append(numeric_column(col, dtype=tf.dtypes.int64))


In [10]:
#train
train_tf_x = []
for col in FEATURES:
    train_tf_x.append(train[col])
    
train_x = np.asarray(train_tf_x)
train_x = np.transpose(train_x)

train_y = np.asarray(train['click'])

#train
test_tf_x = []
for col in FEATURES:
    test_tf_x.append(test[col])
    
test_x = np.asarray(test_tf_x)
test_x = np.transpose(test_x)

test_y = np.asarray(test['click'])

print('Training features:', train_x.shape)
print('Training labels:', train_y.shape)
print('Test features:', test_x.shape)
print('Test labels:', test_y.shape)

Training features: (3347, 21)
Training labels: (3347,)
Test features: (837, 21)
Test labels: (837,)


In [11]:
# Define the model by specifying a neural network layers

# LOOK INTO CLASS WEIGHTS

model = tf.keras.Sequential([
    
    #input layer 1
    tf.keras.layers.Dense(19,input_shape=(len(FEATURES),),activation=tf.nn.tanh),
    
    # Layer 2
#     tf.keras.layers.Dense(16, activation=tf.nn.sigmoid),
    
    # Layer 2
    tf.keras.layers.Dense(12, activation=tf.nn.tanh),
    
    # output
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) 
])

# Set model's settings, including loss function, optimizer, and metrics

model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.fit(train_x, train_y,epochs=10, batch_size=10000, shuffle=True)

# loss: 0.4559 - acc: 0.8300 - when using all sigmoids and 4 layers
# loss: 0.4698 - acc: 0.8300 - when using all sigmoid and decreasing number of nodes
# loss: 0.4584 - acc: 0.8300 - when using all sigmoid and there is only 3 layers 

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
3347/3347 [==============================] - 0s 66us/sample - loss: 0.5179 - acc: 0.8291
Epoch 2/10
3347/3347 [==============================] - 0s 1us/sample - loss: 0.5107 - acc: 0.8291
Epoch 3/10
3347/3347 [==============================] - 0s 2us/sample - loss: 0.5041 - acc: 0.8294
Epoch 4/10
3347/3347 [==============================] - 0s 1us/sample - loss: 0.4984 - acc: 0.8294
Epoch 5/10
3347/3347 [==============================] - 0s 1us/sample - loss: 0.4935 - acc: 0.8294
Epoch 6/10
3347/3347 [==============================] - 0s 1us/sample - loss: 0.4896 - acc: 0.8294
Epoch 7/10
3347/3347 [==============================] - 0s 2us/sample - loss: 0.4860 - acc: 0.8294
Epoch 8/10
3347/3347 [==============================] - 0s 3us/sample - loss: 0.4828 - acc: 0.8294
Epoch 9/10
3347/3347 [==============================] - 0s 2us/sample - loss: 0.4798 - acc: 0.8297
Epoch 10/10
3347/3347 [==============================] - 0s 

In [13]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensor